<a id="table_of_content"></a>

# Part 1: Bag-of-Words Exercises

In the following, we will convert our corpus of 10-k documents into bag-of-words, and convert them into document vectors using Term-Frequency-Inverse-Document-Frequency (tf-idf) re-weighting. Afterward, we will compute sentiments and similarity metrics. Since we will be reusing our notebook, so the various sections are linked below:

1. <a href="#bag_of_word">Compute bag-of-words </a>: implement `bag_of_words` that converts tokenized words into a dictionary of word-counts

2. <a href="#sentiment">Sentiments </a>: using wordlists, compute positive and negative sentiments from bag-of-words. Implement `get_sentiment`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`

# Part 2: Document-Vector Exercises

3. <a href="#compute_idf">Compute idf </a>: computing the inverse document frequency, implement `get_idf`

4. <a href="#compute_tf">Compute tf </a>: computing the term frequency, implement `get_tf`

5. <a href="#doc_vector">Document vector </a>: using the functions `get_idf` and `get_tf`, compute a word_vector for each document in the corpus
6. <a href="#similarity">Similarities </a>: comparing two vectors, and compute cosine and jacard similarity metrics. Implement `get_cos` and `get_jac`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`


## 0. Initialization

First we read in our 10-k documents

In [1]:
# download some excerpts from 10-K files
from download10k import get_files

CIK = {'ebay': '0001065088', 'apple':'0000320193', 'sears': '0001310067'}
get_files(CIK['ebay'], 'EBAY')
get_files(CIK['apple'], 'AAPL')
get_files(CIK['sears'], 'SHLDQ')


# get a list of all 10-ks in our directory
files=! ls *10k*.txt
print("10-k files: ",files)
files = [open(f,"r").read() for f in files]

downloading 10-Ks item 1A for CIK = 0001065088 ...
downloading 10-Ks item 1A for CIK = 0000320193 ...
downloading 10-Ks item 1A for CIK = 0001310067 ...
10-k files:  ['AAPL_10k_2013.txt', 'AAPL_10k_2014.txt', 'AAPL_10k_2015.txt', 'AAPL_10k_2016.txt', 'AAPL_10k_2017.txt', 'EBAY_10k_2013.txt', 'EBAY_10k_2014.txt', 'EBAY_10k_2015.txt', 'EBAY_10k_2016.txt', 'EBAY_10k_2017.txt']


here we define useful functions to tokenize the texts into words, remove stop-words, and lemmatize and stem our words

In [2]:
import numpy as np

# for nice number printing
np.set_printoptions(precision=3, suppress=True)

# tokenize and clean the text
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
# tokenize anything that is not a number and not a symbol
word_tokenizer = RegexpTokenizer(r'[^\d\W]+')

nltk.download('stopwords')
nltk.download('wordnet')


sno = SnowballStemmer('english')
wnl = WordNetLemmatizer()

# get our list of stop_words
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
# add some extra stopwords
stop_words |= {"may", "business", "company", "could", "service", "result", "product", 
               "operation", "include", "law", "tax", "change", "financial", "require",
               "cost", "market", "also", "user", "plan", "actual", "cash", "other",
               "thereto", "thereof", "therefore"}

# useful function to print a dictionary sorted by value (largest first by default)
def print_sorted(d, ascending=False):
    factor = 1 if ascending else -1
    sorted_list = sorted(d.items(), key=lambda v: factor*v[1])
    for i, v in sorted_list:
        print("{}: {:.3f}".format(i, v))

# convert text into bag-of-words
def clean_text(txt):
    lemm_txt = [ wnl.lemmatize(wnl.lemmatize(w.lower(),'n'),'v') \
                for w in word_tokenizer.tokenize(txt) if \
                w.isalpha() and w not in stop_words ]
    return [ sno.stem(w) for w in lemm_txt if w not in stop_words and len(w) > 2 ]

corpus = [clean_text(f) for f in files]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id="bag_of_words"></a>

## 1. Bag-of-Words

Implement a function that converts a list of tokenized words into bag-of-words, i.e. a dictionary that outputs the frequency count of a word

** python already provide the `collections.Counter` class to perform this, but I encourage you to implement your own function as an exercise

<a href="#table_of_content">back to top</a>

In [3]:
from collections import defaultdict

def bag_of_words(words):
    bag = defaultdict(int)
    for w in words:
        bag[w] += 1
    return bag

<a id="sentiment"></a>

## 2. Sentiments
Count the fraction of words that appear in a wordlist, returning a sentiment score between 0 and 1:

$$
\textrm{score} = \frac{\textrm{Number of words in document matching wordlist}}{\textrm{Number of words in document}}
$$

Implement the score in a function `get_sentiment(words, wordlist)`, where words is a list of words. Feel free to use the previous `bag_of_words` function. 
(*for extra challenge, try to code the function in one-line*)

Here, I've included a positive and negative wordlist that I constructed by hand. Due to copyright issues, we are not able to provide other commonly used wordlists. I encourage you to try out different wordlists on your own.

<a href="#table_of_content">back to top</a>

In [5]:
# load wordlist first
import pickle

with open('positive_words.pickle', 'rb') as f:
    positive_words = pickle.load(f)
    # also need to stem and lemmatize the text
    positive_words = set(clean_text(" ".join(positive_words)))
    
with open('negative_words.pickle', 'rb') as f:
    negative_words = pickle.load(f)
    negative_words = set(clean_text(" ".join(negative_words)))
    
# check out the list
# print("positive words: ", positive_words)
# print("negative words: ", negative_words)

In [6]:
def get_sentiment(txt, wordlist):
    matching_words = [ w for w in txt if w in wordlist ]
    return len(matching_words)/len(txt)

In [7]:
# test your function
positive_sentiments = np.array([ get_sentiment(c, positive_words) for c in corpus ])
print(positive_sentiments)

negative_sentiments = np.array([ get_sentiment(c, negative_words) for c in corpus ])
print(negative_sentiments)

[ 0.034  0.033  0.032  0.032  0.031  0.035  0.034  0.033  0.034  0.032]
[ 0.056  0.053  0.052  0.052  0.053  0.056  0.051  0.05   0.054  0.061]


**before continuing part 2, go through the lesson material first!**

<a id="compute_idf"></a>

# Part 2 Document Vector Exercises

## 3. Computer idf
Given a corpus, or a list of bag-of-words, we want to compute for each word $w$, the inverse-document-frequency, or ${\rm idf}(w)$. This can be done in a few steps:

1. Gather a set of all the words in all the bag-of-words (python set comes in handy, and the union operator `|` is useful here)


2. Loop over each word $w$, and compute ${\rm df}_w$, the number of documents where this word appears at least once. A dictionary is useful for keeping track of ${\rm df}_w$


3. After computing ${\rm df}_w$, we can compute ${\rm idf}(w)$. There are usually two possibilities, the simplest one is 
$${\rm idf}(w)=\frac{N}{{\rm df}_w}$$
Where $N$ is the total number of documents in the corpus and $df_w$ the number of documents that contain the word $w$. Frequently, a logarithm term is added as well
$${\rm idf}(w)=\log\frac{N}{{\rm df}_w}$$
One issue with using the logarithm is that when ${\rm df}_w = N$, ${\rm idf}(w)=0$, indicating that words common to all documents would be ignored. If we don't want this behavior, we can define ${\rm idf}(w)=\log\left(1+N/{\rm df}_w\right)$ or ${\rm idf}(w)=1+\log\left(N/{\rm df}_w\right)$ instead. For us, we'll not use the extra +1 for ${\rm idf}$.

In the following, define a function called `get_idf(corpus, include_log=True)` that computes ${\rm idf}(w)$ for all the words in a corpus, where `corpus` for us is a processed list of bag-of-words (stemmed and lemmatized). The optional parameter `include_log` includes the logarithm in the computation.

<a href="#table_of_content">back to top</a>

In [14]:
# compute idf
from math import *

def get_idf(corpus, include_log=True):
    N = len(corpus)
    freq = defaultdict(int)
    words = set()
    for c in corpus:
        words |= set(c)
        
    for w in words:
        freq[w] = sum([ w in c for c in corpus])

    if include_log:
        return { w:log(N/freq[w]) for w in freq }
    else:
        return { w:N/freq[w] for w in freq }

You should expect to see many idf values = 0! This is by design, because we have ${\rm idf}(w)=\log N_d/{\rm df}_w$ and $N_d/{\rm df}_w=1$ for the most common words!

In [15]:
# test your code
idf=get_idf(corpus)
print_sorted(idf, ascending=True)

arrang: 0.000
partner: 0.000
buy: 0.000
within: 0.000
outsourc: 0.000
well: 0.000
equiti: 0.000
onlin: 0.000
safeti: 0.000
singl: 0.000
patent: 0.000
stringent: 0.000
key: 0.000
condit: 0.000
describ: 0.000
greater: 0.000
reflect: 0.000
uniqu: 0.000
perman: 0.000
vari: 0.000
part: 0.000
carrier: 0.000
use: 0.000
hedg: 0.000
varieti: 0.000
primari: 0.000
write: 0.000
give: 0.000
sourc: 0.000
period: 0.000
provis: 0.000
import: 0.000
debt: 0.000
volum: 0.000
continu: 0.000
investor: 0.000
indic: 0.000
advantag: 0.000
digit: 0.000
trademark: 0.000
risk: 0.000
pay: 0.000
channel: 0.000
alleg: 0.000
local: 0.000
recoveri: 0.000
margin: 0.000
denomin: 0.000
legisl: 0.000
liquid: 0.000
environ: 0.000
brand: 0.000
sole: 0.000
unabl: 0.000
avail: 0.000
frequent: 0.000
damag: 0.000
process: 0.000
outcom: 0.000
make: 0.000
attempt: 0.000
altern: 0.000
disput: 0.000
short: 0.000
across: 0.000
charg: 0.000
generat: 0.000
protect: 0.000
manag: 0.000
natur: 0.000
liabil: 0.000
extens: 0.000
facil: 0.

<a id="compute_tf"></a>

## 4. Compute tf

Below we will compute ${\rm tf}(w,d)$, or the term frequency for a given word $w$, in a given document $d$. Since our ultimate goal is to compute a document vector, we'd like to keep a few things in mind

1. Store the ${\rm tf}(w, d)$ for each word in a document as a dictionary
2. Even when a word doesn't appear in the document $d$, we still want to keep a $0$ entry in the dictionary. This is important when we convert the dictionary to a vector, where zero entries are important


There are multiple definitions for ${\rm tf}(w,d)$, the simplest one is

$$
{\rm tf}(w,d)=\frac{f_{w,d}}{a_d}
$$

where $f_{w,d}$ is the number of occurence of the word $w$ in the document $d$, and $a$ the average occurence of all the words in that document for normalization. Just like ${\rm idf}(w)$, a logarithm can be added

$$
{\rm tf}(w,d)=\begin{cases}
\frac{1+\log f_{w,d}}{1+\log a_d} & f_{w,d} > 0 \\
0 & f_{w,d} = 0 \\
\end{cases}
$$

Implement the function `get_df(txt, include_log=True)` that computes ${\rm tf}(w,d)$ for each word in the document (returns a defaultdict(int), so that when supplying words not in the document the dictionary will yield zero instead of an error). Also include the optional parameter `include_log` that enables the additional logarithm term in the computation. I suggest also adding a function called `_tf` that computes the function above. 

<a href="#table_of_content">back to top</a>

In [12]:
import numpy as np
from math import *

def _tf(freq, avg, include_log=True):
    if include_log:
        return 0 if freq == 0 else (1+log(freq))/(1+log(avg))
    else:
        return freq/avg

def get_tf(txt, include_log=True):
    freq = bag_of_words(txt)
    avg = np.mean(list(freq.values()))
    tf = {w:_tf(f,avg, include_log) for w,f in freq.items()}
    return defaultdict(int, tf)

In [13]:
tfs = [ get_tf(c) for c in corpus ]
print_sorted(tfs[0])

affect: 2.146
oper: 2.103
advers: 2.088
condit: 2.080
materi: 2.080
compon: 1.930
develop: 1.918
price: 1.894
third: 1.882
risk: 1.856
parti: 1.856
signific: 1.828
new: 1.782
certain: 1.782
limit: 1.782
manufactur: 1.766
custom: 1.749
applic: 1.749
continu: 1.731
softwar: 1.731
current: 1.693
futur: 1.693
addit: 1.672
subject: 1.672
demand: 1.651
compet: 1.651
time: 1.651
avail: 1.651
foreign: 1.651
credit: 1.628
sale: 1.628
competit: 1.628
assur: 1.628
effect: 1.604
valu: 1.579
increas: 1.579
technolog: 1.579
margin: 1.579
relat: 1.579
distribut: 1.579
key: 1.552
invest: 1.552
mani: 1.552
provid: 1.552
carrier: 1.552
resel: 1.552
retail: 1.552
intern: 1.552
currenc: 1.523
substanti: 1.523
network: 1.523
manag: 1.523
econom: 1.492
depend: 1.492
industri: 1.492
channel: 1.492
devic: 1.492
content: 1.492
term: 1.492
negat: 1.459
supplier: 1.459
competitor: 1.459
patent: 1.459
iphon: 1.459
vendor: 1.459
regul: 1.459
perform: 1.422
sell: 1.422
system: 1.422
obtain: 1.422
receiv: 1.422
asso

<a id="doc_vector"></a>

## 5. Document Vector
Combine the implementation for ${\rm tf}(w,d)$ and ${\rm idf}(w)$ to compute a word-vector for each document in a corpus. Don't forget the zero-padding that is needed when a word appears in some document but not others. 

Implmenet the function `get_vectors(tf,idf)`, the input is the output computed by `get_tf` and `get_idf`

(*optional challenge: implement in one line!*)

<a href="#table_of_content">back to top</a>

In [16]:
def get_vector(tf, idf):
    # TO DO
    return np.array([ tf[w]*idf[w] for w in idf ])

In [17]:
# test your code
doc_vectors = [ get_vector(tf, idf) for tf in tfs ]

for v in doc_vectors:
    print(v)

[ 0.     0.     0.093 ...,  0.093  0.     0.   ]
[ 0.     0.289  0.093 ...,  0.093  0.     0.   ]
[ 0.     0.287  0.092 ...,  0.092  0.     0.   ]
[ 0.     0.286  0.092 ...,  0.092  0.     0.   ]
[ 0.     0.286  0.092 ...,  0.     0.     0.   ]
[ 0.     0.329  0.106 ...,  0.     0.     0.   ]
[ 0.     0.     0.117 ...,  0.117  0.     0.   ]
[ 0.     0.     0.114 ...,  0.187  0.     0.   ]
[ 0.     0.     0.    ...,  0.072  0.     0.   ]
[ 0.    0.    0.   ...,  0.08  0.    0.  ]


<a id ="similarity"></a>

## 6. Similarity

Given two word-vectors $\vec u$ (or $u_i$) and $\vec v$ (or $v_i$), corresponding to two documents, we want to compute different similarity metrics. 

1. Cosine similarity, defined by 
$$
{\rm Sim}_{\cos} = \frac{\vec u \cdot \vec v}{|\vec u| |\vec v|}
$$

2. Jaccard similarity, defined by
$$
{\rm Sim}_{\rm Jaccard} = \frac{\sum_i \min\{u_i, v_i\}}{\sum_i \max\{u_i, v_i\}}
$$

Implement the function similarity as `sim_cis(u,v)` and `sim_jac(u,v)`. Utilize the numpy functions `numpy.linalg.norm` to compute norm of a vector and `np.dot` for computing dot-products. `np.minimum` and `np.maximum` are also useful vectorized pair-wise minimum and maximum functions.

(*optional challenge: implement both functions in one line!*)

<a href="#table_of_content">back to top</a>

In [18]:
from numpy.linalg import norm

def sim_cos(u,v):
    # TO DO
    return np.dot(u,v)/(norm(u)*norm(v))

def sim_jac(u,v):
    # TO DO
    return np.sum(np.minimum(u,v))/np.sum(np.maximum(u,v))

In [19]:
# test your co
# compute all the pairwise similarity metrics
size = len(doc_vectors)
matrix_cos = np.zeros((size,size))
matrix_jac = np.zeros((size,size))

for i in range(size):
    for j in range(size):
        u = doc_vectors[i]
        v = doc_vectors[j]
        matrix_cos[i][j] = sim_cos(u,v)
        matrix_jac[i][j] = sim_jac(u,v)
        
print("Cosine Similarity:")
print(matrix_cos)

print()
print("Jaccard Similarity:")
print(matrix_jac)

Cosine Similarity:
[[ 1.     0.315  0.3    0.29   0.279  0.056  0.071  0.07   0.058  0.058]
 [ 0.315  1.     0.951  0.931  0.902  0.026  0.028  0.028  0.032  0.024]
 [ 0.3    0.951  1.     0.982  0.892  0.027  0.032  0.033  0.036  0.028]
 [ 0.29   0.931  0.982  1.     0.905  0.029  0.034  0.035  0.037  0.031]
 [ 0.279  0.902  0.892  0.905  1.     0.032  0.037  0.037  0.043  0.035]
 [ 0.056  0.026  0.027  0.029  0.032  1.     0.5    0.468  0.368  0.32 ]
 [ 0.071  0.028  0.032  0.034  0.037  0.5    1.     0.864  0.541  0.42 ]
 [ 0.07   0.028  0.033  0.035  0.037  0.468  0.864  1.     0.55   0.403]
 [ 0.058  0.032  0.036  0.037  0.043  0.368  0.541  0.55   1.     0.636]
 [ 0.058  0.024  0.028  0.031  0.035  0.32   0.42   0.403  0.636  1.   ]]

Jaccard Similarity:
[[ 1.     0.296  0.284  0.277  0.274  0.043  0.044  0.043  0.047  0.048]
 [ 0.296  1.     0.901  0.878  0.84   0.021  0.022  0.021  0.029  0.027]
 [ 0.284  0.901  1.     0.972  0.847  0.023  0.024  0.024  0.032  0.03 ]
 [ 0.277  

### Good Job! You've finished all the exercises!

Here is an optional bonus challenge. We often need to debug other people's code to figure out what's wrong. It's particularly difficult when the code doesn't give errors but computes the wrong quantity. Can you describe why the following implementations for some of the exercises above may be wrong? Highlight the words at the bottom to reveal the solutions!

In [20]:
def get_idf_wrong(corpus, include_log=True):
    freq = defaultdict(int)
    for c in corpus:
        for w in c:
            freq[w] += 1
        
    N = len(corpus)
    if include_log:
        return { w:log(N/freq[w]) for w in freq }
    else:
        return { w:N/freq[w] for w in freq }


def get_sentiment_wrong(txt, wordlist):
    matching_words = [ w for w in wordlist if w in txt ]
    return len(matching_words)/len(txt)

def get_vectors_wrong(tf, idf):
    return np.array([ tf[w]*idf[w] for w in tf ])

# Solutions

Drag your mouse over the white space below this cell, and you'll see details about the solutions.  Or, if it's easier, just double-click on the white space below this cell, which will reveal the cell with hidden text.  Also, please check out the file [bagofwords_solutions.py](bagofwords_solutions.py).

<font color="white">
Solution

get_idf: the defaultdict freq here computes the total number of occurences in all the document. We only want to count it once when a word appears in a document. This may lead to a document frequency larger than N, leading to a negative idf! 

get_sentiment_wrong: if a word w appears twice in the document, it won't be counted properly!

get_vectors_wrong: tf may not contain all the words in idf. We need zero padding for words that appear in idf but not in tf! 
</font> 